<a href="https://colab.research.google.com/github/SuperSpawn/Side-Projects/blob/main/MeNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
actions = ['eat', 'sleep', 'meet someone new', 'drink water']

In [ ]:
import torch
from torch import nn
import torch.optim as optim

# Define the network
class MeNN(nn.Module):
    def __init__(self, activation, input_size, third_size, output_size):
        super(MeNN, self).__init__()
        self.fc1 = nn.Linear(input_size, third_size)
        self.activation = activation
        # self.fc2 = nn.Linear(second_size, third_size)
        self.fc3 = nn.Linear(third_size, output_size)
        self.double()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        # x = self.fc2(x)
        # x = self.activation(x)
        x = self.fc3(x)
        return x

In [ ]:
import csv

def save_dataset(output_file, data):
  with open(output_file, "w", newline="") as csvfile:
    # Create a CSV writer object
    writer = csv.DictWriter(csvfile, fieldnames=["input", "output"])

    # Write the header row
    writer.writeheader()

    # Write each data point as a row in the CSV file
    for item in data:
        writer.writerow(item)



In [ ]:
# Create data

# Generate data
# save it to a file
# load file for training
import random
import numpy as np

def generate_instance(ranges):
  instance = []
  for inp in ranges:
    val = random.uniform(inp[0], inp[1])
    instance.append(val)
  return instance

def generate_dataset_case(input_ranges, output_ranges, size):
    dataset = []
    for _ in range(size):
      data = []
      data.append(generate_instance(input_ranges))
      data.append(generate_instance(output_ranges))
      dataset.append(data)
    return dataset

def generate_dataset(size, flags, case_size):
  total_data = []
  for i in range(size):
    input_ranges = np.full((size, 2), (0.5, 0.5))
    output_ranges = np.full((size, 2), (0.5, 0.5))

    input_ranges_low = input_ranges.copy()
    input_ranges_high = input_ranges.copy()
    output_ranges_low = output_ranges.copy()
    output_ranges_high = output_ranges.copy()

    input_ranges_low[i] = (0.0, 0.5)
    input_ranges_high[i] = (0.5, 1.0)

    output_ranges_low[i] = (0.0, 0.5)
    output_ranges_high[i] = (0.5, 1.0)

    if flags[i]['reverse']:
      temp = output_ranges_low
      output_ranges_low = output_ranges_high
      output_ranges_high = temp

    dataset_case_low = generate_dataset_case(input_ranges_low, output_ranges_low, case_size)
    dataset_case_high = generate_dataset_case(input_ranges_high, output_ranges_high, case_size)

    dataset_case = [i for i in dataset_case_low] + dataset_case_high

    total_data = [i for i in dataset_case] + total_data
  return total_data


# print(dataset)
# save_dataset('test.csv', dataset)

In [ ]:
dataset = generate_dataset(4, [{'reverse': False}, {'reverse': False}, {'reverse': True}, {'reverse': False}], 4000)
print(len(dataset))

32000


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import csv
import numpy as np

class MeNNDataset(Dataset):
    def __init__(self, data):
        self.data = np.array(data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return a single data point and its label
        return self.data[idx]



In [ ]:
# Create datasets and data loaders
train_dataset = MeNNDataset(dataset)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_dataset = MeNNDataset("test.csv")
# test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
for data in train_loader:
  print(data[0])

In [ ]:
# Define the model, loss function, and optimizer
model = MeNN(nn.ReLU(), 4, 4, 4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Train the model
for epoch in range(40):
    # Train on each batch
    for data in train_loader:
        for i in data:
          # Forward pass
          output = model(i[0])

          # Calculate loss
          loss = criterion(output, i[1])

          # Backward pass and update parameters
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

In [ ]:
# Load the model
model.eval()

# Convert the input to a PyTorch tensor
input_tensor = torch.tensor([0.1, 0.0, 0.8, 0.9], dtype=torch.float64)

# Make a prediction
prediction = model(input_tensor)

print(prediction)


_, topk_indices = torch.topk(prediction, k=1)
action = topk_indices[0]
print(actions[action])

tensor([ 0.5386, -0.3085,  0.1372,  0.7362], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
drink water


In [ ]:
test_data = generate_dataset(4, [{'reverse': False}, {'reverse': False}, {'reverse': True}, {'reverse': False}], 100)
print(len(test_data))

800


In [ ]:
hit = 0
total = 0
for data in test_data:
  input_tensor = torch.tensor(data[0], dtype=torch.float64)
  prediction = model(input_tensor) # input

  _, topk_indices = torch.topk(prediction, k=1)
  prediction_index = topk_indices[0]

  max_value = max(data[1])
  max_index = data[1].index(max_value)

  if max_index == prediction_index:
    hit+=1
  total+=1

print(f"Accuracy: %{100 * hit/total}")

Accuracy: %84.875
